In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Hide/Display Code"></form>''')

# Import Modules

In [2]:
!pip install ipywidgets > /dev/null
!jupyter nbextension enable --py widgetsnbextension > /dev/null

import os
from tqdm import tqdm
from IPython.display import Image, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
# from anacinx_misc import *
%run -i anacinx_misc.py

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
def create_instance(home_dir, image_path):
    #!apptainer instance start -H /home/bbogale/results/ dir_anacinx/ anacin
    cmd = 'apptainer instance start -H {} {} anacinx_jupyter_instance'.format(home_dir, image_path)
    #!apptainer instance start -H home_dir image_path instance_name
    !{cmd}

def kill_instance():
    !apptainer instance stop anacinx_jupyter_instance

def clean_output_dir(output_dir):
    #implement error checking
    if output_dir == "":
        return 0
    os.system("rm -rf {}/*".format(output_dir))
    return 1

def trace_execution(executable_path, args, benchmark_type, num_processes, num_runs, num_iterations, pnmpi_conf, step_start, step_size, step_end, message_size, output_dir):
    gen_name = ""
    if benchmark_type == "message_race":
        gen_name = "naive_reduce"
    elif benchmark_type == "amg2013":
        gen_name = "amg2013"
    #For benchmark
    #Add functionality to choose message type
#     !apptainer exec instance://anacin_jupyter_instance \
#     python3 /ANACIN-X/apps/comm_pattern_generator/config/json_gen.py \
#     "naive_reduce" 512 5 "/home/bbogale/results/" 0.0 0.1 1.0
    
    !apptainer exec instance://anacinx_jupyter_instance \
    python3 /ANACIN-X/apps/comm_pattern_generator/config/json_gen.py \
    {gen_name} {message_size} {num_iterations} "{output_dir}/" {step_start} {step_size} {step_end}
#      "naive_reduce" {message_size} {num_iterations} "{output_dir}/" {step_start} {step_size} {step_end}

    !apptainer exec instance://anacinx_jupyter_instance cp  /ANACIN-X/submodules/CSMPI/config/default_glibc.json {output_dir}
    
    for i in tqdm(range(0,int(num_runs))):
        #Create run directory
        run_dir = "{}/run_{}/".format(output_dir, str(i).rjust(3, "0"))
        !mkdir {run_dir}
        
        #Copy CSMPI config file to run directories
        !cp {output_dir}/default_glibc.json {run_dir}/

        !sed -i 's|null|'{run_dir}'|g' {run_dir}/default_glibc.json
        
        !cd {run_dir} ; mpirun -np {num_processes} apptainer exec \
        --env LD_PRELOAD=/ANACIN-X/submodules/PnMPI/build/lib/libpnmpi.so \
        --env PNMPI_LIB_PATH=/ANACIN-X/anacin-x/pnmpi/patched_libs \
        --env PNMPI_CONF=/ANACIN-X/anacin-x/pnmpi/configs/{pnmpi_conf} \
        --env CSMPI_CONFIG={run_dir}/default_glibc.json \
        instance://anacinx_jupyter_instance {executable_path} {args} > /dev/null
        
   
    #Delete CSMPI conf from output dir
    !rm {output_dir}/default_glibc.json
    
def generate_event_graph(num_processess, num_runs, dumpi_config,  output_dir):
    #Come back to this later and add config selection functionality
    dumpi_to_graph_bin="/ANACIN-X/submodules/dumpi_to_graph/build/dumpi_to_graph"
    #dumpi_to_graph_config="/ANACIN-X/submodules/dumpi_to_graph/config/" + {dumpi_config}
    dumpi_to_graph_config="/ANACIN-X/submodules/dumpi_to_graph/config/"

    #dumpi_to_graph_config="/ANACIN-X/submodules/dumpi_to_graph/config/dumpi_and_csmpi.json"

    
    for i in tqdm(range(0, num_runs)):
        run_dir = "{}/run_{}/".format(output_dir, str(i).rjust(3, "0"))
        !cd {run_dir} ; \
        mpirun -np {num_processess} apptainer exec instance://anacinx_jupyter_instance {dumpi_to_graph_bin} {dumpi_to_graph_config}{dumpi_config} {run_dir} > /dev/null
    
    #!mpirun -np {num_processess} {dumpi_to_graph_bin} {dumpi_to_graph_config} {run_dir}
    
def extract_slices(num_processess, num_runs, slicing_policy_config, output_dir):
    #Add functionality to select custom slicing policies
    #slicing_policy="/ANACIN-X/anacin-x/event_graph_analysis/slicing_policies/barrier_delimited_full.json"
    #slicing_policy="/ANACIN-X/anacin-x/event_graph_analysis/slicing_policies/" + {slicing_policy_config}
    slicing_policy="/ANACIN-X/anacin-x/event_graph_analysis/slicing_policies/"

    
    for i in tqdm(range(0, num_runs)):
        run_dir = "{}/run_{}/".format(output_dir, str(i).rjust(3, "0"))
        !cd {run_dir} ; mpirun -np {num_processess} \
        apptainer exec instance://anacinx_jupyter_instance \
        /ANACIN-X/anacin-x/event_graph_analysis/extract_slices.py {run_dir}/"event_graph.graphml" {slicing_policy}{slicing_policy_config} -o "slices" > /dev/null
            
def compute_kdts(num_processess, slicing_policy_config, output_dir):
    #Consult Jack about this
    graph_kernel_policy = "/ANACIN-X/anacin-x/event_graph_analysis/graph_kernel_policies/wlst_5iters_logical_timestamp_label.json"
    #slicing_policy="/ANACIN-X/anacin-x/event_graph_analysis/slicing_policies/barrier_delimited_full.json"
    slicing_policy="/ANACIN-X/anacin-x/event_graph_analysis/slicing_policies/"
    
    !cd {output_dir} ; \
    mpirun -np {num_processess} apptainer exec instance://anacinx_jupyter_instance \
    /ANACIN-X/anacin-x/event_graph_analysis/compute_kernel_distance_time_series.py \
    {output_dir} {graph_kernel_policy} --slicing_policy {slicing_policy}{slicing_policy_config} -o "kdts.pkl" --slice_dir_name {output_dir}/"slices" -c > /dev/null
        
def create_graph(output_dir, step_start, step_size, step_end):
    !apptainer exec instance://anacinx_jupyter_instance \
    bash -c 'python3 /ANACIN-X/anacin-x/event_graph_analysis/visualization/make_message_nd_plot.py \
    {output_dir}/kdts.pkl \
    message_race \
    /ANACIN-X/anacin-x/event_graph_analysis/graph_kernel_policies/wlst_5iters_logical_timestamp_label.json \
    {output_dir}/kdts \
    {step_start} {step_size} {step_end}'
        
#     0.0 0.1 1.0'
    
    
    


# Image and Output Paths

In [4]:
display(output_dir_widget)

Text(value='', description='Enter output directory path:', layout=Layout(width='50%'), style=DescriptionStyle(…

In [5]:
display(image_path_widget)

Text(value='', description='Enter container image path:', layout=Layout(width='50%'), style=DescriptionStyle(d…

# Create Instance

In [6]:
display(create_instance_widget)

Button(button_style='success', description='Create Instance', layout=Layout(width='50%'), style=ButtonStyle())

Creating Instance..
INFO:    instance started successfully


<!-- ### Demonstration
- We will follow each step of the execution process:
    1. Trace execution
    2. Event graph generation
    3. Slice extraction
    2. Visualization -->

# Type and Parameter Selection

In [7]:
display(widgets.HBox([benchmark_type_selector_widget, extern_type_selector_widget]))

Dropdown(description='Benchmark Application:', index=1, layout=Layout(width='50%'), options=(('', ''), ('Messa…

IntText(value=0, description='Number of processess:', layout=Layout(width='50%'), style=DescriptionStyle(descr…

IntText(value=0, description='Number of runs:', layout=Layout(width='50%'), style=DescriptionStyle(description…

IntText(value=0, description='Number of iterations:', layout=Layout(width='50%'), style=DescriptionStyle(descr…

Dropdown(description='PnMPI Config:', layout=Layout(width='50%'), options=(('', ''), ('csmpi', 'csmpi.conf'), …

Text(value='', description='Starting Step', layout=Layout(width='50%'), style=DescriptionStyle(description_wid…

Text(value='', description='Step Size', layout=Layout(width='50%'), style=DescriptionStyle(description_width='…

Text(value='', description='Step End', layout=Layout(width='50%'), style=DescriptionStyle(description_width='i…

Text(value='', description='Message Size (bytes)', layout=Layout(width='50%'), style=DescriptionStyle(descript…

# Execution Trace Collection

In [8]:
display(trace_widget)

Button(description='Trace', layout=Layout(width='50%'), style=ButtonStyle())

Tracing...


  0%|                                                                                                                                                                    | 0/2 [00:00<?, ?it/s]

In [9]:
print(param_benchmark_type)

# Event Graph Generation

In [10]:
display(gen_event_graph_widget)

Button(description='Generate Event Graph', layout=Layout(width='50%'), style=ButtonStyle())

# Slices Extraction

In [11]:
display(slice_extraction_widget)

Button(description='Extract Slices', layout=Layout(width='50%'), style=ButtonStyle())

# KDTS Computation

In [12]:
display(compute_kdts_widget)

Button(description='Compute KDTS', layout=Layout(width='50%'), style=ButtonStyle())

# Viualization

In [13]:
display(create_visualization_widget)

Button(description='Create Visualization', layout=Layout(width='50%'), style=ButtonStyle())

In [14]:
display(display_visualization_widget)

Button(description='Display Visualization', layout=Layout(width='50%'), style=ButtonStyle())

# Kill

In [15]:
display(kill_instance_widget)


Button(button_style='danger', description='Kill Instance', layout=Layout(width='50%'), style=ButtonStyle())